In [90]:
import pandas as pd
import numpy as np
import sys,os
this_dir = os.path.abspath('.')
parent_dir = os.path.abspath('..')
if this_dir not in sys.path:
    sys.path.append(parent_dir)
    sys.path.append(this_dir)
import db_info#@UnresolvedImport
import typing
import plotly_utilities as pu
import datetime

In [2]:
opttab = 'sec_schema.options_table'
futtab = 'sec_schema.underlying_table'

In [3]:
StrList = typing.List[str]
class SqlDownloader():
    def __init__(self,config_name:str='local'):
        self.pga = db_info.get_db_info(config_name=config_name)
    
    def get_sql(
        self,
        columns=None,
        table=futtab,
        where_clauses=None,
        groupby_columns=None,
        orderby_columns=None,
        limit=None
    ):
        cols = '*' if columns is None else ','.join(columns)
        sql = f"select {cols} from {table}"
        
        if where_clauses is not None:
            wc_string = f'where {where_clauses[0]}'
            if len(where_clauses)>1:
                wc_string =  wc_string + " and " + ' and '.join(where_clauses[1:])
            wc_string = wc_string.replace('%','%%')
            sql = sql + " " + f"{wc_string}"
        if groupby_columns is not None:
            sql += f" group by {','.join(groupby_columns)}"
        if orderby_columns is not None:
            sql += f" order by {','.join(orderby_columns)}"
        if limit is not None:
            sql += f" limit {int(str(limit))}"
        df = self.pga.get_sql(sql)
        return df
        
    def get_options(self,symbol:str,yyyymmdd_beg:int=None,yyyymmdd_end:int=None):
        sql = f"select * from {opttab} where symbol='{symbol}'"
        if yyyymmdd_beg is not None:
            sql += f' and settle_date>={yyyymmdd_beg}'
        if yyyymmdd_end is not None:
            sql += f' and settle_date<={yyyymmdd_end}'
        df =  self.pga.get_sql(sql)
        if df is not None and len(df)>2:
            df = df.sort_values(['pc','settle_date','strike'])
        return df

    def get_futures(self,symbol_list:StrList,yyyymmdd_beg:int=None,yyyymmdd_end:int=None):
        if type(symbol_list) is list or type(symbol_list) is np.ndarray:
            sym_string = ",".join([f"'{s}'" for s in symbol_list]) 
        else:
            sym_string = f"'{symbol_list}'"
        sql = f"select * from {futtab} where symbol in ({sym_string})"
        if yyyymmdd_beg is not None:
            sql += f' and settle_date>={yyyymmdd_beg}'
        if yyyymmdd_end is not None:
            sql += f' and settle_date<={yyyymmdd_end}'
        df =  self.pga.get_sql(sql)
        if df is not None and len(df)>2:
            df = df.sort_values(['symbol','settle_date'])
        return df


In [4]:
sqld = SqlDownloader(config_name='local')

  sec_db


In [22]:
df_fut = sqld.get_futures('NGH15')
# df_fut[df_fut.close==2.662]

In [25]:
df_opt = sqld.get_options('NGH15')
# df_opt[(df_opt.settle_date==20201218) & (df_opt.pc=='C')].sort_values('strike')

In [26]:
df_opt.settle_date.min(),df_opt.settle_date.max()

(20110606, 20150224)

In [56]:
wc_list = [
    "symbol='CLZ12'",
    "pc='C'",
    "strike=80",
    "settle_date>20120501"
]
df_sql = sqld.get_sql(table=opttab,where_clauses=wc_list)
df_sql

,symbol,strike,pc,settle_date,open,high,low,close,adj_close,volume,open_interest
0,CLZ12,80.0,C,20120502,0.00,27.39,27.22,27.39,27.39,0,10032
1,CLZ12,80.0,C,20120503,0.00,25.38,25.38,25.38,25.38,0,10032
2,CLZ12,80.0,C,20120504,0.00,21.92,21.45,21.92,21.92,0,10032
3,CLZ12,80.0,C,20120507,0.00,21.45,20.78,21.45,21.45,0,10032
4,CLZ12,80.0,C,20120508,0.00,20.76,19.90,20.76,20.76,0,10032
...,...,...,...,...,...,...,...,...,...,...,...
132,CLZ12,80.0,C,20121107,0.00,4.76,4.76,4.76,4.76,0,11885
133,CLZ12,80.0,C,20121108,0.00,5.23,5.23,5.23,5.23,2064,10880
134,CLZ12,80.0,C,20121109,0.00,6.11,6.11,6.11,6.11,354,10946
135,CLZ12,80.0,C,20121112,0.00,5.58,5.58,5.58,5.58,1225,10946


In [57]:
df_sql.settle_date.unique()

array([20120502, 20120503, 20120504, 20120507, 20120508, 20120509,
       20120510, 20120511, 20120514, 20120515, 20120516, 20120517,
       20120518, 20120521, 20120522, 20120523, 20120524, 20120525,
       20120529, 20120530, 20120531, 20120601, 20120604, 20120605,
       20120606, 20120607, 20120608, 20120611, 20120612, 20120613,
       20120614, 20120615, 20120618, 20120619, 20120620, 20120621,
       20120622, 20120625, 20120626, 20120627, 20120628, 20120629,
       20120702, 20120703, 20120705, 20120706, 20120709, 20120710,
       20120711, 20120712, 20120713, 20120716, 20120717, 20120718,
       20120719, 20120720, 20120723, 20120724, 20120725, 20120726,
       20120727, 20120730, 20120731, 20120801, 20120802, 20120803,
       20120806, 20120807, 20120808, 20120809, 20120810, 20120813,
       20120814, 20120815, 20120816, 20120817, 20120820, 20120821,
       20120822, 20120823, 20120824, 20120827, 20120828, 20120829,
       20120830, 20120831, 20120904, 20120905, 20120906, 20120

In [60]:
cols = ['distinct symbol']
wc_list = [
    "symbol like 'CL%'",
]
sqld.get_sql(table=futtab,columns=cols,where_clauses=wc_list)

,symbol
0,CLM17
1,CLN11
2,CLV20
3,CLG15
4,CLK15
...,...
238,CLK22
239,CLN18
240,CLN13
241,CLK25


### Compare a CSO download from Reuters Eikon to the spread components.
`way0c1p.csv` contains history data for the CLH21/CLJ21 0 strike CSO Put with the spread history.

In [88]:
df_way0c1p = pd.read_csv('./way0c1p.csv')
df_way0c1p['settle_date'] = df_way0c1p.Date.apply(lambda v:datetime.datetime.strptime(v,'%d-%b-%Y'))
df_way0c1p['settle_date'] = df_way0c1p.settle_date.apply(lambda v:int(v.year*100*100+v.month*100+v.day))
df_way0c1p = df_way0c1p[['WAY0O1P','settle_date']].sort_values('settle_date')
df_clh21 = sqld.get_futures('CLH21')
df_way0c1p = df_way0c1p.merge(df_clh21[['settle_date','close']],on='settle_date',how='inner').rename(columns={'WAY0O1P':'opt_price','close':'CLH21'})
df_clj21 = sqld.get_futures('CLJ21')
df_way0c1p = df_way0c1p.merge(df_clj21[['settle_date','close']],on='settle_date',how='inner').rename(columns={'close':'CLJ21'})
df_way0c1p['spread'] = df_way0c1p.CLJ21 - df_way0c1p.CLH21


In [89]:
pu.iplt(
    pu.plotly_plot(
        df_way0c1p[['settle_date','spread','opt_price']],
        x_column='settle_date',
        y_left_label='CSO/Spread Price',
        plot_title="CLH21/CLJ21 CSO Zero Put vs Calendar Spread"
    )
)

In [4]:
#!jupyter nbconvert secdb_example.ipynb --to python